# Experiment 01

- Loss function: ArcFace
- SwimB
- closed set

In [1]:
import os
import sys
import pandas as pd
import torch
from torch.utils.data import DataLoader

In [2]:
sys.path.append('..')

from src.dataset import SeaTurtleDataset

In [3]:
# INDIVIDUALS = 10
# TRAIN_ENCOUNTERS = 2
# TEST_ENCOUNTERS = 1

BATCH_SIZE = 8

In [5]:
dataset_dir='../data'

train_csv_path = os.path.join(dataset_dir, "metadata_splits_filtered_closed_train.csv")
test_csv_path = os.path.join(dataset_dir, "metadata_splits_filtered_closed_test.csv")

train_dataset = SeaTurtleDataset(annotations_file=train_csv_path, img_dir=dataset_dir)
test_dataset = SeaTurtleDataset(annotations_file=test_csv_path, img_dir=dataset_dir)


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)